In [6]:
# Install required libraries
!pip install pandas plotly ipywidgets --quiet

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# ===== Load data from public GitHub raw URL =====
url = "https://raw.githubusercontent.com/leonism/sample-superstore/master/data/superstore.csv"
df = pd.read_csv(url, encoding='latin1', low_memory=False)

# ===== Preprocess =====
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df['YearMonth'] = df['Order Date'].dt.to_period('M')

# ===== Basic Info =====
print("DataFrame shape:", df.shape)
display(df.head())

# ===== KPIs =====
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum() if 'Profit' in df.columns else None
total_orders = df.shape[0]

print(f"\nTotal Sales: ${total_sales:,.2f}")
if total_profit is not None:
    print(f"Total Profit: ${total_profit:,.2f}")
print(f"Total Orders: {total_orders}")

# ===== Initial Visualizations =====
if 'Region' in df.columns:
    region_sales = df.groupby('Region')['Sales'].sum().reset_index()
    fig_region = px.bar(region_sales, x='Region', y='Sales',
                        title='Revenue by Region', color='Region')
    fig_region.show()

if 'Product Name' in df.columns:
    top_products = df.groupby('Product Name')['Sales'].sum().nlargest(10).reset_index()
    fig_products = px.bar(top_products, x='Product Name', y='Sales',
                          title='Top 10 Products by Sales')
    fig_products.show()

# ===== Interactive Dashboard Filters =====
regions = df['Region'].dropna().unique().tolist() if 'Region' in df.columns else []
categories = df['Category'].dropna().unique().tolist() if 'Category' in df.columns else []

region_dropdown = widgets.Dropdown(options=['All'] + regions, description='Region:')
category_dropdown = widgets.Dropdown(options=['All'] + categories, description='Category:')

def update_dashboard(region, category):
    temp = df.copy()
    if region != 'All' and 'Region' in temp.columns:
        temp = temp[temp['Region'] == region]
    if category != 'All' and 'Category' in temp.columns:
        temp = temp[temp['Category'] == category]

    ts = temp['Sales'].sum()
    tp = temp['Profit'].sum() if 'Profit' in temp.columns else None
    to = temp.shape[0]

    print(f"\nTotal Sales: ${ts:,.2f}" + (f" | Total Profit: ${tp:,.2f}" if tp is not None else "") + f" | Orders: {to}")

    if 'Region' in temp.columns:
        rs = temp.groupby('Region')['Sales'].sum().reset_index()
        fig1 = px.bar(rs, x='Region', y='Sales', title='Revenue by Region', color='Region')
        fig1.show()
    if 'Product Name' in temp.columns:
        tp10 = temp.groupby('Product Name')['Sales'].sum().nlargest(10).reset_index()
        fig2 = px.bar(tp10, x='Product Name', y='Sales', title='Top 10 Products by Sales')
        fig2.show()

widgets.interactive(update_dashboard, region=region_dropdown, category=category_dropdown)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00
DataFrame shape: (10800, 22)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,YearMonth
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2.0,0.00,41.9136,2017-11
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3.0,0.00,219.5820,2017-11
2,3,CA-2017-138688,2017-06-12,6/16/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2.0,0.00,6.8714,2017-06
3,4,US-2016-108966,2016-10-11,10/18/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5.0,0.45,-383.0310,2016-10
4,5,US-2016-108966,2016-10-11,10/18/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2.0,0.20,2.5164,2016-10



Total Sales: $2,297,200.86
Total Profit: $286,397.02
Total Orders: 10800


interactive(children=(Dropdown(description='Region:', options=('All', 'South', 'West', 'Central', 'East'), val…